In [ ]:
# !pip install tf-nightly-gpu
!pip install conda
!pip install pandas
!pip install numpy
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# https://drive.google.com/file/d/1AHI8M7OLFJzgw3TM9d8i_U9ojE7UlcoY/view?usp=sharing
# https://drive.google.com/file/d/1SMvX03NhzR4qGdSYtS2i14OlRNt5v5iQ/view?usp=sharing
# var url = ""
# function getIdFromUrl(url) { return url.match(/[-\w]{25,}/); }
# getIdFromUrl(url)?

downloaded_train_data = drive.CreateFile({'id':"1AHI8M7OLFJzgw3TM9d8i_U9ojE7UlcoY"}) 
downloaded_train_data.GetContentFile('house_train.csv') 

downloaded_test_data = drive.CreateFile({'id':"1SMvX03NhzR4qGdSYtS2i14OlRNt5v5iQ"})
downloaded_test_data.GetContentFile('house_test.csv')   

print(downloaded_train_data)
print(downloaded_test_data)


from sklearn.model_selection import cross_val_score, GridSearchCV
import warnings
warnings.filterwarnings('ignore')
# %tensorflow_version 2.
import tensorflow as tf
# device_name = tf.test.gpu_device_name()

# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')

GoogleDriveFile({'id': '1AHI8M7OLFJzgw3TM9d8i_U9ojE7UlcoY', 'kind': 'drive#file', 'etag': '"MTU3NjQ3NDQxNjAwMA"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1AHI8M7OLFJzgw3TM9d8i_U9ojE7UlcoY', 'webContentLink': 'https://drive.google.com/uc?id=1AHI8M7OLFJzgw3TM9d8i_U9ojE7UlcoY&export=download', 'alternateLink': 'https://drive.google.com/file/d/1AHI8M7OLFJzgw3TM9d8i_U9ojE7UlcoY/view?usp=drivesdk', 'embedLink': 'https://drive.google.com/file/d/1AHI8M7OLFJzgw3TM9d8i_U9ojE7UlcoY/preview?usp=drivesdk', 'iconLink': 'https://drive-thirdparty.googleusercontent.com/16/type/text/csv', 'title': 'house_train.csv', 'mimeType': 'text/csv', 'labels': {'starred': False, 'hidden': False, 'trashed': False, 'restricted': False, 'viewed': True}, 'copyRequiresWriterPermission': False, 'createdDate': '2020-12-12T22:16:13.249Z', 'modifiedDate': '2019-12-16T05:33:36.000Z', 'modifiedByMeDate': '2019-12-16T05:33:36.000Z', 'lastViewedByMeDate': '2020-12-12T22:16:13.249Z', 'markedViewedByMeDate': '1970

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
train_data = pd.read_csv('house_train.csv')
test_data = pd.read_csv('house_test.csv')

### **second code**

In [ ]:
print(len(train_data.columns))
train_missing = train_data.isna().sum() / train_data.shape[0]
train_missing[train_missing > 0.7]
test_missing = test_data.isna().sum() / test_data.shape[0]
test_missing[test_missing > 0.7]
train_data.drop(train_missing[train_missing > 0.7].index, axis = 1, inplace = True)
test_data.drop(test_missing[test_missing > 0.7].index, axis = 1, inplace = True)
print(len(train_data.columns))

81
77


In [ ]:
train_data['LotFrontage'].interpolate(axis=0, inplace=True)
train_data[['MasVnrType']].fillna('None', inplace=True)
train_data.dropna(subset=['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond'], inplace=True)
train_data.dropna(subset=['BsmtQual', 'BsmtCond', 'BsmtFinType1'], inplace=True)
train_data.dropna(subset=['MasVnrType', 'MasVnrArea'], inplace=True)
train_data.drop('FireplaceQu', axis=1, inplace=True)
train_data['BsmtExposure'].fillna('No', inplace=True)
train_data['BsmtFinType2'].fillna('Unf', inplace=True)
train_data['Electrical'].fillna('SBrkr', inplace=True)

test_data.fillna(method='ffill', inplace=True)
test_data.fillna(method='bfill', inplace=True)


KeyError: ignored

In [ ]:
# Copy Train data excluding target
trainData_Copy = train_data.drop(['SalePrice', 'Id'], axis=1).copy()
testData_Copy = test_data.drop('Id', axis=1).copy()
# Combine Train and test for One Hot Encoding
combined_Data = pd.concat([trainData_Copy,testData_Copy], keys = [0,1])
# Do One Hot Encoding for categorical features
combined_Data = pd.get_dummies(combined_Data)
# Separate Train data and test data
X_train = combined_Data.xs(0)
X_test = combined_Data.xs(1)
y_train = train_data["SalePrice"]


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
search_random = {'n_estimators': range(1000, 5000, 100), 'learning_rate':[0.01, 0.1, 0.05]}
abc = GradientBoostingRegressor()
abc.fit(X_train, y_train)
abc_predict = abc.predict(X_test)

In [ ]:
# X_test = testData_Copy
model.fit(X_train,y_train)
prediction = model.predict(X_test)
output = pd.DataFrame({'Id': test_data.Id, 'SalePrice': prediction.astype(int)})
output.to_csv('my_submission_sub2_house.csv', index=False)
from google.colab import files
files.download('my_submission_sub2_house.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### **first code**

In [ ]:
from sklearn.preprocessing import LabelEncoder

name = []
value = []
for r in train_data.columns:
  name.append(r)
  value.append(len(train_data.groupby(r)))
df = pd.DataFrame({'name':name, 'value':value})
# print(df.value.sort_values(ascending=False).head(60))
# print(len(train_data.columns))
for i in (df.value.sort_values(ascending=False).keys()):
  name = df['name'][i]
  value = df['value'][i]
  if train_data[name].dtypes == 'object':
    train_data[name].fillna((train_data[name]).mode()[0], inplace=True)
    keyf = 1
    for s in (train_data[name].unique()):
      train_data[name] = train_data[name].replace(s, keyf)
      keyf = keyf+1
  elif train_data[name].dtypes == 'int64' or train_data[name].dtypes == 'float64':
    if value > 25:
      train_data[name].fillna(train_data[name].median(), inplace=True)
      train_data[(name + '_5')] = pd.qcut(train_data['LotArea'], 5)
      label = LabelEncoder()
      train_data[(name + '_5_Code')] = label.fit_transform(train_data[(name + '_5')])
      train_data = train_data.drop([(name + '_5')], axis=1)
# print(len(train_data.columns))
# print(train_data.columns)
train_data.head(3)


name = []
value = []
for r in test_data.columns:
  name.append(r)
  value.append(len(test_data.groupby(r)))
df = pd.DataFrame({'name':name, 'value':value})
# print(df.value.sort_values(ascending=False).head(60))
# print(len(test_data.columns))
for i in (df.value.sort_values(ascending=False).keys()):
  name = df['name'][i]
  value = df['value'][i]
  if test_data[name].dtypes == 'object':
    test_data[name].fillna((test_data[name]).mode()[0], inplace=True)
    keyf = 1
    for s in (test_data[name].unique()):
      test_data[name] = test_data[name].replace(s, keyf)
      keyf = keyf+1
  elif test_data[name].dtypes == 'int64' or test_data[name].dtypes == 'float64':
    if value > 25:
      test_data[name].fillna(test_data[name].median(), inplace=True)
      test_data[(name + '_5')] = pd.qcut(test_data['LotArea'], 5)
      label = LabelEncoder()
      test_data[(name + '_5_Code')] = label.fit_transform(test_data[(name + '_5')])
      test_data = test_data.drop([(name + '_5')], axis=1)
# print(len(test_data.columns))
# print(test_data.columns)
test_data.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Id_5_Code,LotArea_5_Code,GrLivArea_5_Code,BsmtUnfSF_5_Code,1stFlrSF_5_Code,TotalBsmtSF_5_Code,BsmtFinSF1_5_Code,GarageArea_5_Code,2ndFlrSF_5_Code,MasVnrArea_5_Code,WoodDeckSF_5_Code,OpenPorchSF_5_Code,BsmtFinSF2_5_Code,EnclosedPorch_5_Code,LotFrontage_5_Code,YearBuilt_5_Code,GarageYrBlt_5_Code,ScreenPorch_5_Code,YearRemodAdd_5_Code,MiscVal_5_Code
0,1461,20,1,80.0,11622,1,1,1,1,1,1,1,1,1,1,1,1,5,6,1961,1961,1,1,1,1,1,0.0,1,1,1,1,1,1,1,468.0,1,144.0,270.0,882.0,1,...,1,1.0,730.0,1,1,1,140,0,0,0,120,0,1,1,1,0,6,2010,1,1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
1,1462,20,2,81.0,14267,1,1,2,1,1,2,1,1,2,1,1,1,6,6,1958,1958,2,1,2,2,2,108.0,1,1,1,1,1,1,2,923.0,2,0.0,406.0,1329.0,1,...,1,1.0,312.0,1,1,1,393,36,0,0,0,0,1,1,2,12500,6,2010,1,1,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
2,1463,60,2,74.0,13830,1,1,2,1,1,1,1,2,2,1,1,2,5,5,1997,1998,1,1,1,1,1,0.0,1,1,2,2,1,1,3,791.0,2,0.0,137.0,928.0,1,...,2,2.0,482.0,1,1,1,212,34,0,0,0,0,1,1,1,0,3,2010,1,1,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier

X_train = train_data.drop(['SalePrice','Id','SalePrice_5_Code'], axis=1)
y_train = train_data['SalePrice']

model = RandomForestClassifier(n_estimators=200, random_state=2)
model.fit(X_train,y_train)
importance = pd.DataFrame({'feature':X_train.columns, 'importance': np.round(model.feature_importances_,3)})
importance = importance.sort_values('importance', ascending=False).set_index('feature')

type(importance)
for i in importance.iterrows():
  print(i)

('GrLivArea', importance    0.034
Name: GrLivArea, dtype: float64)
('TotalBsmtSF', importance    0.032
Name: TotalBsmtSF, dtype: float64)
('BsmtUnfSF', importance    0.032
Name: BsmtUnfSF, dtype: float64)
('1stFlrSF', importance    0.032
Name: 1stFlrSF, dtype: float64)
('LotArea', importance    0.031
Name: LotArea, dtype: float64)
('GarageArea', importance    0.031
Name: GarageArea, dtype: float64)
('LotFrontage', importance    0.029
Name: LotFrontage, dtype: float64)
('GarageYrBlt', importance    0.028
Name: GarageYrBlt, dtype: float64)
('YearBuilt', importance    0.028
Name: YearBuilt, dtype: float64)
('MoSold', importance    0.027
Name: MoSold, dtype: float64)
('YearRemodAdd', importance    0.026
Name: YearRemodAdd, dtype: float64)
('BsmtFinSF1', importance    0.026
Name: BsmtFinSF1, dtype: float64)
('OpenPorchSF', importance    0.023
Name: OpenPorchSF, dtype: float64)
('WoodDeckSF', importance    0.021
Name: WoodDeckSF, dtype: float64)
('Neighborhood', importance    0.021
Name: Nei

In [ ]:
# with tf.device('/device:GPU:0'):
final = ['1stFlrSF_5_Code','1stFlrSF','2ndFlrSF_5_Code','2ndFlrSF','3SsnPorch','Alley','BedroomAbvGr','BldgType','BsmtCond','BsmtExposure','BsmtFinSF1_5_Code','BsmtFinSF1','BsmtFinSF2_5_Code','BsmtFinSF2','BsmtFinType1','BsmtFinType2','BsmtFullBath','BsmtHalfBath','BsmtQual','BsmtUnfSF_5_Code','BsmtUnfSF','CentralAir','Condition1','Condition2','Electrical','EnclosedPorch_5_Code','EnclosedPorch','ExterCond','Exterior1st','Exterior2nd','ExterQual','Fence','FireplaceQu','Fireplaces','Foundation','FullBath','Functional','GarageArea_5_Code','GarageArea','GarageCars','GarageCond','GarageFinish','GarageQual','GarageType','GarageYrBlt_5_Code','GarageYrBlt','GrLivArea_5_Code','GrLivArea','HalfBath','Heating','HeatingQC','HouseStyle','Id_5_Code','KitchenAbvGr','KitchenQual','LandContour','LandSlope','LotArea_5_Code','LotArea','LotConfig','LotFrontage_5_Code','LotFrontage','LotShape','LowQualFinSF','MasVnrArea_5_Code','MasVnrArea','MasVnrType','MiscFeature','MiscVal','MoSold','MSSubClass','MSZoning','Neighborhood','OpenPorchSF_5_Code','OpenPorchSF','OverallCond','OverallQual','PavedDrive','PoolArea','PoolQC','RoofMatl','RoofStyle','SaleCondition','SaleType','ScreenPorch_5_Code','ScreenPorch','Street','TotalBsmtSF_5_Code','TotalBsmtSF','TotRmsAbvGrd','Utilities','WoodDeckSF_5_Code','WoodDeckSF','YearBuilt_5_Code','YearBuilt','YearRemodAdd_5_Code','YearRemodAdd','YrSold']
grid_param = {
'n_estimators': [100, 200, 300, 500],
'criterion':['gini', 'entropy'],
'min_samples_split': [1, 2, 10, 20],
'min_samples_leaf': [ 2, 5],
'bootstrap': [True, False],
}
# grid_param = {
# 'n_estimators': [100],
# 'criterion':['gini'],
# 'min_samples_split': [2],
# 'min_samples_leaf': [2],
# 'bootstrap': [True],
# }

gd_sr = GridSearchCV(estimator=model,param_grid=grid_param,scoring='accuracy',cv=5,n_jobs=-1)
gd_sr.fit(X_train[final], y_train)
best_parameters = gd_sr.best_params_
print(best_parameters)


{'bootstrap': False, 'criterion': 'gini', 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 100}


In [ ]:
# {'bootstrap': True, 'criterion': 'entropy', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
# {'bootstrap': False, 'criterion': 'gini', 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 100}
model = RandomForestClassifier(n_estimators=100, bootstrap=False, criterion= 'gini', min_samples_leaf=5, min_samples_split=2, random_state=2)

model1 = model
all_accuracies = cross_val_score(estimator=model1, X=X_train, y=y_train, cv=5)
print(all_accuracies)
print(all_accuracies.mean())

[0.01369863 0.01027397 0.01027397 0.02054795 0.01712329]
0.014383561643835616


In [ ]:
# s = test_data.isnull().sum()
# print(s)
# for i in s.iteritems():
#     print(i)

test_data['BsmtFullBath'].fillna(test_data['BsmtFullBath'].median(), inplace=True)
test_data['BsmtHalfBath'].fillna(test_data['BsmtHalfBath'].median(), inplace=True)
test_data['GarageCars'].fillna(test_data['GarageCars'].median(), inplace=True)

In [ ]:
X_test = test_data[final]
model.fit(X_train[final],y_train)
prediction = model.predict(X_test)
output = pd.DataFrame({'Id': test_data.Id, 'SalePrice': prediction.astype(int)})
output.to_csv('my_submission_house.csv', index=False)
from google.colab import files
files.download('my_submission_house.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>